<a href="https://colab.research.google.com/github/guptaharsh-in/SMACrossover_Strategy_Backtesting_Class/blob/main/SMACrossover_Strategy_Backtesting_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install library nsepy if necessary
#pip install nsepy

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nsepy as nse
from datetime import date

In [ ]:
# Create SMACrossover Class
class SMACrossover():
  def __init__(self, symbol,SMA_S, SMA_L, start, end, index):
    self.symbol = symbol
    self.SMA_S= SMA_S
    self.SMA_L= SMA_L
    self.start=start
    self.end=end
    self.index=index
    self.result= None
    self.get_data()

# Create Get Data Function 

  def get_data(self):
    df = nse.get_history(self.symbol,start=self.start,end=self.end,index=self.index)
    data= df.Close.to_frame()

#Calcutate Log Returns and Simple Moving Averages (Short and Long)
    data["returns"] = np.log(data.Close.div(data.Close.shift(1)))
    data["SMA_S"] = data.Close.rolling(self.SMA_S).mean()
    data["SMA_L"] = data.Close.rolling(self.SMA_L).mean()
    data.dropna(inplace=True)
    self.data2 =data

    return data

# Create Test Results Function
  def test_results(self):
    data=self.data2.copy().dropna()

#Calcutate Buy and Hold Returns and Drawdown Columns
    data["cumret"] = data.returns.cumsum().apply(np.exp)
    data["cummax"] = data.cumret.cummax()
    data['drawdownperc'] = (data['cummax'] - data['cumret'])/(data['cummax'])
    data.dropna(inplace=True)

#Calcutate Strategy Returns and Drawdown Columns
    data["position"] = np.where(data["SMA_S"]>data["SMA_L"],1,-1)
    data["strategy"] = data["returns"]*data.position.shift(1)
    data["cumret_str"] = data.strategy.cumsum().apply(np.exp)
    data["cummax_str"] = data.cumret_str.cummax()
    data['drawdownperc_str'] = (data['cummax_str'] - data['cumret_str'])/(data['cummax_str'])
    data.dropna(inplace=True)
    
#Calcutate Strategy Returns and Drawdown
    ret = np.exp(data["strategy"].sum())
    std = data["strategy"].std()*np.sqrt(252)
    draw_str = data.drawdownperc_str.max()
    drawdate_str = data.drawdownperc_str.idxmax()
  
#Calcutate Buy and Hold Returns and Drawdown
    retbnh = np.exp(data["returns"].sum())
    stdbnh = data["returns"].std()*np.sqrt(252)
    drawbnh = data.drawdownperc.max()
    drawdatebnh = data.drawdownperc.idxmax()

#Calcutate Performance and Outperformance
    perf = data["cumret_str"].iloc[-1]
    outperf = perf - data["cumret"].iloc[-1]
    self.results=data

#Return Results
    return print(f"Performance = {perf},\nOutperformance = {outperf},\n\nStrategy Return={ret},\nStrategy Risk={std},\nMax Drawdown Strategy={draw_str},\nMax Drawdown Date Strategy={drawdate_str}\n\nBuyandHold Return={retbnh},\nBuyandHold Risk={stdbnh},\nMax Drawdown BuyandHold={drawbnh},\nMax Drawdown Date BuyandHold={drawdatebnh}")


#Create Plot Results Function
  def plot_results(self):
    title= f"Symbol:{self.symbol} | SMA_S:{self.SMA_S} | SMA_L:{self.SMA_L}"
    self.results[["cumret","cumret_str","position"]].plot(title=title, figsize= (12,8), xlabel= "Date", ylabel = "Returns")
    self.results[["Close", "SMA_S","SMA_L"]].plot(title=title, figsize= (12,8),xlabel= "Date", ylabel = "Price")
    

In [ ]:
#Load Data Syntax Example
#test = SMACrossover("NIFTY", 50, 200, date(2020,1,1), date(2023,1,1), True)

In [ ]:
#Test and Plot Results Syntax
#test.test_results()
#test.plot_results()